In [45]:
import argparse
import gym
from vans_gym.envs import VansEnvsSeq
from vans_gym.solvers import CirqSolverR, Checker
from vans_gym.models import DuelDQN
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import tensorflow as tf

In [2]:
solver = CirqSolverR(n_qubits = 2, observable_name="Ising_",qlr=0.05,qepochs=100)
checker = Checker(solver)

env = VansEnvsSeq(solver,checker=checker, depth_circuit=3)


In [3]:
model = DuelDQN(env, name="WorkingOut",
               priority_scale=0.1, learning_rate=0.01, tau=0.5)
model.replay_buffer.load("DuelDQN/run_9/data_collected/")

In [14]:
state = model.env.reset()
done = False
while not done:
    action = model.give_action(state, ep=0)
    next_state, reward, done, info = model.env.step(action)
    state = next_state


In [65]:
[1,1] == [1,1]

True

In [71]:
for k in model.replay_buffer.states[:model.replay_buffer.count]:
    if not (k[0] == k[1])[0]:
        print(k)
    #if k[0] != k[1]:
    #    print("differentss")

[[-1. -1. -1.]
 [ 4. -1. -1.]]
[[-1. -1. -1.]
 [ 1. -1. -1.]]
[[-1. -1. -1.]
 [ 3. -1. -1.]]
[[-1. -1. -1.]
 [ 0. -1. -1.]]
[[-1. -1. -1.]
 [ 1. -1. -1.]]
[[-1. -1. -1.]
 [ 3. -1. -1.]]
[[-1. -1. -1.]
 [ 4. -1. -1.]]
[[-1. -1. -1.]
 [ 5. -1. -1.]]
[[-1. -1. -1.]
 [ 5. -1. -1.]]
[[-1. -1. -1.]
 [ 4. -1. -1.]]
[[-1. -1. -1.]
 [ 2. -1. -1.]]
[[-1. -1. -1.]
 [ 2. -1. -1.]]
[[-1. -1. -1.]
 [ 2. -1. -1.]]
[[-1. -1. -1.]
 [ 3. -1. -1.]]
[[-1. -1. -1.]
 [ 4. -1. -1.]]
[[-1. -1. -1.]
 [ 2. -1. -1.]]
[[-1. -1. -1.]
 [ 4. -1. -1.]]
[[-1. -1. -1.]
 [ 0. -1. -1.]]
[[-1. -1. -1.]
 [ 0. -1. -1.]]
[[-1. -1. -1.]
 [ 5. -1. -1.]]
[[-1. -1. -1.]
 [ 1. -1. -1.]]
[[-1. -1. -1.]
 [ 4. -1. -1.]]
[[-1. -1. -1.]
 [ 3. -1. -1.]]
[[-1. -1. -1.]
 [ 2. -1. -1.]]
[[-1. -1. -1.]
 [ 1. -1. -1.]]
[[-1. -1. -1.]
 [ 5. -1. -1.]]
[[-1. -1. -1.]
 [ 2. -1. -1.]]
[[-1. -1. -1.]
 [ 1. -1. -1.]]
[[-1. -1. -1.]
 [ 3. -1. -1.]]
[[-1. -1. -1.]
 [ 4. -1. -1.]]
[[-1. -1. -1.]
 [ 3. -1. -1.]]
[[-1. -1. -1.]
 [ 1. -1. -1.]]
[[-1. -1

In [72]:
state = model.env.reset()
while not done:
    action = model.give_action(state, ep=model.schedule(k, total_timesteps))
    next_state, reward, done, info = model.env.step(action)

In [53]:
model.replay_buffer.states[np.where(np.round(model.replay_buffer.rewards)==1)][:4]

array([[[1., 5., 3.],
        [1., 5., 3.]],

       [[3., 5., 3.],
        [3., 5., 3.]],

       [[2., 5., 3.],
        [2., 5., 3.]],

       [[4., 2., 4.],
        [4., 2., 4.]]], dtype=float32)

OK! Let's train the network s.t. it recognizes the optimal action at first ..

In [39]:
for k in tqdm(range(100)):
    model.learn_step(batch_size=64)

100%|██████████| 100/100 [00:06<00:00, 15.24it/s]


In [42]:
(states, actions, rewards, nstates, dones), importance, indices = model.replay_buffer.get_minibatch(batch_size=1, priority_scale=model.priority_scale)

In [52]:
model.prim_qnet.predict(tf.stack(np.array([[-1,-1,-1]]).astype(np.float32)))

array([[180.22615, 160.75142, 171.45552, 185.59444, 207.60086, 228.84091]],
      dtype=float32)

In [54]:
model.prim_qnet.predict(tf.stack(np.array([[-1,-1,-1]]).astype(np.float32)))

array([[180.22615, 160.75142, 171.45552, 185.59444, 207.60086, 228.84091]],
      dtype=float32)

In [55]:
model.prim_qnet.predict(tf.stack(np.array([[3.,-1,-1]]).astype(np.float32)))

array([[187.52869, 182.91484, 191.88638, 184.84996, 189.40388, 194.17445]],
      dtype=float32)